In [1]:

# We will use this cell to import all the packages you will need in the following - think of it as turning on all your systems
# in your cockpit

# This makes sure that if you change code in your external scripts, they will be updated


#import checker
#import generator

from IPython.display import display, clear_output
import numpy as np
import time
import math
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd

#import importlib
#importlib.reload(checker)
#importlib.reload(generator)

# now go ahead and Run the cell. This might take a while...
# while the cell is running, you will see ln[*] next to it. Once it finished, you will see the number of execution
# In case you want to interrupt the run of a cell, press Ctrl + C (on your german keyboard, this is Strg + C)

# pip install tensorflow

# There is this joke that once you have added this import line to your code,
# you are allowed to sell your product telling everyone that you are using AI. Life is easy, sometimes.

import tensorflow as tf
print('Tensorflow version:', tf.__version__, '\n')

# Keras is a model-level library, meaning that it is built upon tensorflow (using it as a backend) - allowing for
# high-level building blocks. Making it even easier to design neural networks.
# We will access it as tf.keras

# The tf and k abbreviations are best practice (same for numpy np and pandas pd),
# since you do not want to type T E N S O R F L O W all over your code.
# They are prevalent all over the industry and academia in a way that you'll risk a fight if you import them differently.

# Loading the MNIST dataset in one line
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train_normalized = x_train/255
x_test_normalized = x_test/255

# in the next step, we also need to reshape our input to fit our input layer later on.
# This is due to keras expecting a definition for how many channels your input sample has, as we
# deal with gray scale this is 1.
x_train= x_train_normalized.reshape(-1, 28, 28, 1)
x_test = x_test_normalized.reshape(-1, 28, 28, 1)

# Printing the shape
print('x_train:', x_train.shape)
print('y_train:', y_train.shape)
print('x_test:', x_test.shape)
print('y_test:', y_test.shape)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Tensorflow version: 2.13.0 

11490434/11490434 [==============================] - 0s 0us/step
x_train: (60000, 28, 28, 1)
y_train: (60000,)
x_test: (10000, 28, 28, 1)
y_test: (10000,)


In [4]:
# This is the moment where you define your model's architecture

pepsi = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=(28,28,1)),
    
    tf.keras.layers.Conv2D(filters=28, kernel_size=3, padding='same', activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.005)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(filters=28, kernel_size=3, padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(filters=28, kernel_size=5, strides=2, padding='same', activation='relu', use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.GaussianNoise(0.75),

    tf.keras.layers.Conv2D(filters=56, kernel_size=3, padding='same', activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.005)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(filters=56, kernel_size=3, padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(filters=56, kernel_size=5, strides=2, padding='same', activation='relu', use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.GaussianNoise(0.75),

    tf.keras.layers.Conv2D(filters=112, kernel_size=4, activation='relu'),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(82, kernel_regularizer = tf.keras.regularizers.l2(0.005), activation = 'relu'),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.GaussianNoise(0.75),

    tf.keras.layers.Dense(54, kernel_regularizer = tf.keras.regularizers.l2(0.005), activation = 'relu'),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.GaussianNoise(0.75),
    tf.keras.layers.Dense(10, activation='softmax', use_bias=False)
    ])

pepsi.summary()

# Define your loss
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# print(-tf.math.log(1/10))

sampleID = 100
loss_fn(y_train[:1], pepsi(x_train[sampleID-1:sampleID]).numpy()).numpy()

# Compiling basically means to prepare the training routine for your model which consists of the optimizer,
# the loss, and the metrics which are to be reported during training

pepsi.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001),
              loss= loss_fn,
              metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 28)        280       
                                                                 
 batch_normalization (Batch  (None, 28, 28, 28)        112       
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 28)        7084      
                                                                 
 batch_normalization_1 (Bat  (None, 28, 28, 28)        112       
 chNormalization)                                                
                                                                 
 conv2d_2 (Conv2D)           (None, 14, 14, 28)        19600     
                                                                 
 batch_normalization_2 (Bat  (None, 14, 14, 28)        1

In [ ]:
import datetime
import os

%load_ext tensorboard
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)


In [3]:
 tf.keras.layers.Conv2D(filters=28, kernel_size=3, padding='same', activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.005)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(filters=28, kernel_size=3, padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(filters=28, kernel_size=5, strides=2, padding='same', activation='relu', use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.GaussianNoise(0.75),

    tf.keras.layers.Conv2D(filters=56, kernel_size=3, padding='same', activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.005)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(filters=56, kernel_size=3, padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(filters=56, kernel_size=5, strides=2, padding='same', activation='relu', use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.GaussianNoise(0.75),

    tf.keras.layers.Conv2D(filters=112, kernel_size=4, activation='relu'),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(82, kernel_regularizer = tf.keras.regularizers.l2(0.005), activation = 'relu'),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.GaussianNoise(0.75),

    tf.keras.layers.Dense(54, kernel_regularizer = tf.keras.regularizers.l2(0.005), activation = 'sigmoid'),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.GaussianNoise(0.75),

IndentationError: unexpected indent (3052013043.py, line 2)

In [ ]:
nets = 50
model = [0] * nets
for j in range(nets):
    
    model[j] = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=(28,28,1)),
    
    tf.keras.layers.Conv2D(filters=28, kernel_size=3, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(filters=28, kernel_size=3, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(filters=28, kernel_size=5, strides=2, padding='same', activation='relu', use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.GaussianNoise(0.75),
        
    tf.keras.layers.Conv2D(filters=56, kernel_size=3, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(filters=56, kernel_size=3, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(filters=56, kernel_size=5, strides=2, padding='same', activation='relu', use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.GaussianNoise(0.75),

    tf.keras.layers.Conv2D(filters=112, kernel_size=4, activation='relu'),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Flatten(),
        
    tf.keras.layers.Dense(82, kernel_regularizer = tf.keras.regularizers.l2(0.005), activation = 'relu'),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.GaussianNoise(0.75),

    tf.keras.layers.Dense(54, kernel_regularizer = tf.keras.regularizers.l2(0.005), activation = 'sigmoid'),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.GaussianNoise(0.75),
        
    tf.keras.layers.Dense(10, activation='softmax', use_bias=False)
    ])
    
    # Define your loss
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    # print(-tf.math.log(1/10))
    sampleID = 100
    loss_fn(y_train[:1], model[j](x_train[sampleID-1:sampleID]).numpy()).numpy()
    
    model[j].compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001),
                    loss= loss_fn,
                    metrics=['accuracy'])
    
#vlr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor = 0.8, patience =10, min_lr=0.000001)
vlr2 = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.975 ** x)

er = tf.keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=15,
        restore_best_weights=True
)

checkpoint_filepath = 'tmp1/model.{val_accuracy:.4f}.h5'
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    save_freq= 'epoch'
)
    
for j in range(nets):

    datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=15,
                zoom_range = 0.15,
                shear_range=0.1,
                width_shift_range=0.1,
                height_shift_range=0.1,
                rescale=0,
                fill_mode = 'nearest',
                horizontal_flip=False,
                vertical_flip=False)
    datagen.fit(x_train)

    history = model[j].fit(
        datagen.flow(x_train, y_train, batch_size = 32),
        epochs=500,
        shuffle = True,
        validation_data=(x_test, y_test),
        callbacks=[vlr2, er],# model_checkpoint, ],
        verbose=1
    )
    
    (test_loss, test_acc) = model[j].evaluate(x_test, y_test)
    print("Loss: ", test_loss)
    print("Accuracy: ", test_acc)
    model_name = f'tmp/model.{test_acc:.4f}.h5'
    model[j].save(model_name, save_format='h5')
  
plt.figure(figsize=(13, 5))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['Train','Test'])
plt.grid()
plt.show()

In [ ]:
# This line would start up tensorboard for you
%tensorboard --logdir logs --host localhost

In [ ]:
# after the training finishes, we will also save Marvin in Keras style (HDF5), so we do not have to
# train him again
# every time we start our computer. Obviously, by changing the model_name, you can also save different
# configurations of Marvin. The name has to be a string, like this: 'name.h5'
model_name = 'tmp/model_99.74'
pepsi.save(model_name, save_format='h5')

# It is best practice to indicate what configuration changes you did within the name, so you know
# which model you need to load already from its name
# Let's say instead of a learning rate of 0.001 you used 0.1, your naming could then look like:
# 'marvin_lr01.h5'

print('Success! You saved Marvin as: ', model_name)

In [12]:
model_name = 'tmp/model.0.9974.h5'
pepsi = tf.keras.models.load_model(model_name)

checkpoint_filepath = 'tmp_re/model.{val_accuracy:.4f}.h5'
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    save_freq= 'epoch'
)

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
            rotation_range=15,
            zoom_range = 0.15,
            #shear_range=0.20,
            width_shift_range=0.1,
            height_shift_range=0.1,
            rescale=0,
            fill_mode = 'nearest',
            horizontal_flip=False,
            vertical_flip=False)
datagen.fit(x_train)

history = pepsi.fit(
    datagen.flow(x_train, y_train, batch_size = 64),
    epochs=50,
    shuffle = True,
    validation_data=(x_test, y_test),
    callbacks=[model_checkpoint, vlr2],
    verbose=1
)

Epoch 1/50
  1/938 [..............................] - ETA: 14:27 - loss: 0.0400 - accuracy: 0.9844

2023-11-26 15:27:52.621111: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_47/dropout_188/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


938/938 [==============================] - 23s 23ms/step - loss: 0.1550 - accuracy: 0.9814 - val_loss: 0.1050 - val_accuracy: 0.9933 - lr: 0.0010
Epoch 2/50
938/938 [==============================] - 21s 22ms/step - loss: 0.1361 - accuracy: 0.9845 - val_loss: 0.1089 - val_accuracy: 0.9907 - lr: 9.5000e-04
Epoch 3/50
938/938 [==============================] - 21s 22ms/step - loss: 0.1369 - accuracy: 0.9844 - val_loss: 0.0995 - val_accuracy: 0.9946 - lr: 9.0250e-04
Epoch 4/50
938/938 [==============================] - 21s 23ms/step - loss: 0.1335 - accuracy: 0.9848 - val_loss: 0.1034 - val_accuracy: 0.9937 - lr: 8.5737e-04
Epoch 5/50
938/938 [==============================] - 21s 23ms/step - loss: 0.1276 - accuracy: 0.9858 - val_loss: 0.0903 - val_accuracy: 0.9947 - lr: 8.1451e-04
Epoch 6/50
707/938 [=====================>........] - ETA: 5s - loss: 0.1234 - accuracy: 0.9857

KeyboardInterrupt: 

In [ ]:
# Plot for the intuitive approach

numbers_to_display = 196
num_cells = math.ceil(math.sqrt(numbers_to_display))
plt.figure(figsize=(15, 15))

for plot_index in range(numbers_to_display):
    predicted_label = predictions[plot_index]
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    color_map = 'Greens' if predicted_label == y_test[plot_index] else 'Reds'
    plt.subplot(num_cells, num_cells, plot_index + 1)
    plt.imshow(x_test_normalized[plot_index].reshape((28, 28)), cmap=color_map)
    plt.xlabel(predicted_label)

plt.subplots_adjust(hspace=1, wspace=0.5)
plt.show()

In [ ]:
# load a saved marvin configuration you want to evaluate
model_name = 'mtmp/model_99.74.h5'
pepsi_reloaded = tf.keras.models.load_model(model_name)

# Let Marvin predict on the test set, so we have some data to evaluate his performance.
predictions = pepsi_reloaded.predict([x_test])
# predictions = pepsi.predict([x_test])

# Remember that the prediction of Marvin is a probability distribution over all ten-digit classes
# We want him to assign the digit class with the highest probability to the sample.
predictions = np.argmax(predictions, axis=1)
#pd.DataFrame(predictions)

In [ ]:

(test_loss, test_acc) = pepsi_reloaded.evaluate(x_test, y_test)
print("Loss: ", test_loss)
print("Accuracy: ", test_acc)

confusion_matrix = tf.math.confusion_matrix(y_test, predictions)

f, ax = plt.subplots(figsize=(9, 7))
sn.heatmap(
    confusion_matrix,
    annot=True,
    linewidths=.7,
    fmt="d",
    square=True,
    ax=ax,
    cmap="viridis",
)
plt.show()